<center>
  <font size="7">Déploiement d'API du modèle</font><br>
  <font size="5">Projet 7 - Implémentez un modèle de scoring</font>
</center>
<div align="right">
  <font size="4"><i>par Jean Vallée</i></font>
</div>

<hr size=5>

L'API REST sert d'interface d'échange de requêtes et reponses avec le modèle 
- est disponible en ligne au développeur et aux utilisateurs
- c'est la classe Python du modèle avec ses méthodes

# Outils de déploiement

- les commandes Git pour la gestion de versions du code et de l’API
- [Github](https://github.com/) pour stocker et partager l’API et assurer une intégration continue
- [Github Actions](https://github.com/JeanRosselVallee/project_7/actions) pour le déploiement en continu de l’API
- Pytest
    - des jeux de tests unitaires sont préparés dans ce notebook
    - des tests unitaires sont exécutés par le Notebook N°5 Test d'API
    - les tests automatisés font partie des jobs du [workflow](https://github.com/JeanRosselVallee/project_7/blob/main/.github/workflows/python-app-test-%26-deploy.yml) de GitHub Actions

# Pré-requis

In [1]:
dir_staging = './staging_model'

In [2]:
! pip install -r $dir_staging/requirements.txt

In [3]:
import mlflow
import numpy as np
import pandas as pd

In [4]:
import json
with open('../config.json') as file_object:
    dict_config = json.load(file_object)

In [5]:
ip_server       = dict_config['ip_host']
user            = dict_config['user']
port_staging    = dict_config['port_staging']

# Serveur de Staging

## Chargement du modèle

In [6]:
! pip install --quiet -r ./staging_model/requirements.txt

In [7]:
staging_model = mlflow.pyfunc.load_model(dir_staging)
staging_model

mlflow.pyfunc.loaded_model:
  flavor: mlflow.sklearn

### Vérification de la signature

La signature du modèle chargé indique les valeurs valides à lui fournir

Attributs

In [8]:
li_features = staging_model.metadata.get_input_schema().input_names()
li_features

['CODE_GENDER_M',
 'EXT_SOURCE_3',
 'EXT_SOURCE_2',
 'NAME_EDUCATION_TYPE_Secondary_or_secondary_special',
 'NAME_EDUCATION_TYPE_Higher_education',
 'NAME_CONTRACT_TYPE_Cash_loans',
 'NAME_INCOME_TYPE_Working']

Cible

In [9]:
staging_model.metadata.get_output_schema().input_names

<bound method Schema.input_names of ['TARGET': long (required)]>

### Vérification de prédiction

On prédit la cible pour une observation issue du fichier des prédictions _True Positive_

In [10]:
path_TP = '../modeling/data/out/X_TP.csv'
df_TP = pd.read_csv(path_TP)

In [11]:
df_TP_sample_1 = df_TP.head(1)
df_TP_sample_1.T

,0
CODE_GENDER_M,1.000000
EXT_SOURCE_3,0.202087
EXT_SOURCE_2,0.580628
NAME_EDUCATION_TYPE_Secondary_or_secondary_special,1.000000
NAME_EDUCATION_TYPE_Higher_education,0.000000
NAME_CONTRACT_TYPE_Cash_loans,1.000000
NAME_INCOME_TYPE_Working,1.000000


In [12]:
print('Predicted target value =', str(staging_model.predict(df_TP_sample_1)))

Predicted target value = [1]


### Préparation des tests unitaires
Dans ce notebook,
- les tests unitaires ne s'y dérouleront pas
- les données destinées aux tests sont sauvegardées dans des fichiers 

In [13]:
dir_test_data = '../test_api/data/'
! mkdir -p $dir_test_data

#### Liste d'attributs
Ce fichier sera utilisé par la page _form.html_

In [14]:
str_li_features = str(li_features)
print(str_li_features)
with open(dir_test_data + '/li_features.txt', "w") as file_form:
    print(str_li_features, file=file_form)

['CODE_GENDER_M', 'EXT_SOURCE_3', 'EXT_SOURCE_2', 'NAME_EDUCATION_TYPE_Secondary_or_secondary_special', 'NAME_EDUCATION_TYPE_Higher_education', 'NAME_CONTRACT_TYPE_Cash_loans', 'NAME_INCOME_TYPE_Working']


#### Liste de types
Ce fichier sera utilisé par la page _form.html_

In [15]:
li_types = staging_model.metadata.get_input_schema().input_types()
str_li_types = str(li_types)
print(str_li_types)
with open(dir_test_data + '/li_types.txt', "w") as file_form:
    print(str_li_types, file=file_form)

[long, double, double, long, long, long, long]


#### Dictionnaire simple d'observations
Ce fichier sera utilisé par le test unitaire de connexion du script _test_1.py_

In [16]:
str_features_values = df_TP_sample_1.to_json(orient='records')
print(str_features_values)
with open(dir_test_data + '/dict_X_single.json', 'w') as file_object:
    print(str_features_values, file=file_object)

[{"CODE_GENDER_M":1,"EXT_SOURCE_3":0.2020866017,"EXT_SOURCE_2":0.5806283659,"NAME_EDUCATION_TYPE_Secondary_or_secondary_special":1,"NAME_EDUCATION_TYPE_Higher_education":0,"NAME_CONTRACT_TYPE_Cash_loans":1,"NAME_INCOME_TYPE_Working":1}]


#### Dictionnaire multiple d'observations
- Ce fichier sera utilisé par les tests unitaires du script _test_2.py_
- Il est obtenu à partir des résultats TP (vrais positifs) des prédictions

In [17]:
nb_observations = 3
df_TP_sample_N = df_TP.tail(nb_observations)
df_TP_sample_N

,CODE_GENDER_M,EXT_SOURCE_3,EXT_SOURCE_2,NAME_EDUCATION_TYPE_Secondary_or_secondary_special,NAME_EDUCATION_TYPE_Higher_education,NAME_CONTRACT_TYPE_Cash_loans,NAME_INCOME_TYPE_Working
2355,0,0.218859,0.588678,1,0,1,1
2356,1,0.510853,0.367941,1,0,1,1
2357,0,0.070109,0.030184,0,1,1,0


In [18]:
file_TP_sample = dir_test_data + 'X_sample.csv'
df_TP_sample_N.to_csv(file_TP_sample)
! wc -l $file_TP_sample 

4 ../test_api/data/X_sample.csv


In [19]:
str_features_values = df_TP_sample_N.to_json(orient='records')
print(str_features_values)
with open(dir_test_data + '/dict_X_sample.json', 'w') as file_object:
    print(str_features_values, file=file_object)

[{"CODE_GENDER_M":0,"EXT_SOURCE_3":0.2188590822,"EXT_SOURCE_2":0.588678411,"NAME_EDUCATION_TYPE_Secondary_or_secondary_special":1,"NAME_EDUCATION_TYPE_Higher_education":0,"NAME_CONTRACT_TYPE_Cash_loans":1,"NAME_INCOME_TYPE_Working":1},{"CODE_GENDER_M":1,"EXT_SOURCE_3":0.5108529062,"EXT_SOURCE_2":0.3679405554,"NAME_EDUCATION_TYPE_Secondary_or_secondary_special":1,"NAME_EDUCATION_TYPE_Higher_education":0,"NAME_CONTRACT_TYPE_Cash_loans":1,"NAME_INCOME_TYPE_Working":1},{"CODE_GENDER_M":0,"EXT_SOURCE_3":0.0701088438,"EXT_SOURCE_2":0.0301835653,"NAME_EDUCATION_TYPE_Secondary_or_secondary_special":0,"NAME_EDUCATION_TYPE_Higher_education":1,"NAME_CONTRACT_TYPE_Cash_loans":1,"NAME_INCOME_TYPE_Working":0}]


## Lancement du serveur

**Arrêt**

In [20]:
mask = ':' + port_staging
! pkill -f "$mask"

**Démarrage**

In [21]:
ip_host = '0.0.0.0'
shell_command = 'nohup mlflow models serve -m ' + dir_staging + ' -p ' + port_staging + ' -h ' + ip_host
get_ipython().system_raw(shell_command + ' --no-conda &')          # runs model API in background

**Vérification d'exécution**

Il y a 2 processus qui tournent par serveur

In [22]:
! ps aux | grep "scoring_server" | grep -v "grep" | awk '{print $2, $15, $19}'

69336 0.0.0.0:5677 mlflow.pyfunc.scoring_server.wsgi:app
69337 0.0.0.0:5677 mlflow.pyfunc.scoring_server.wsgi:app


## Configuration réseau
### Tunnel SSH
Sur le terminal [_Azure CLI_](https://portal.azure.com/#cloudshell/) :

In [23]:
shell_command = 'nohup ssh -v -N -L ' + port_staging + ':localhost:' + port_staging + \
                    ' ' + user + '@' + ip_server + ' &'
print(shell_command)

nohup ssh -v -N -L 5677:localhost:5677 jvisa4031@4.233.201.217 &


### Aperture de ports
Sur le [portail _Azure_](https://portal.azure.com/), _Network Settings_ de la VM : _Create Inbound Port Rule_

In [24]:
print('Destination Port =', port_staging)

Destination Port = 5677


## URL de l'API

In [25]:
url_staging = ip_server + ':' + port_staging + '/invocations'
print('URL Staging    -> http://' + url_staging)

URL Staging    -> http://4.233.201.217:5677/invocations


## Vérification d'accès distant

Demande par requête POST de prédiction de la cible pour une observation

In [26]:
str_features_values = df_TP_sample_1.to_json(orient='split')
print(str_features_values)

{"columns":["CODE_GENDER_M","EXT_SOURCE_3","EXT_SOURCE_2","NAME_EDUCATION_TYPE_Secondary_or_secondary_special","NAME_EDUCATION_TYPE_Higher_education","NAME_CONTRACT_TYPE_Cash_loans","NAME_INCOME_TYPE_Working"],"index":[0],"data":[[1,0.2020866017,0.5806283659,1,0,1,1]]}


In [27]:
str_data = '\'{"dataframe_split": ' + str_features_values + '}\' '
print(str_data)

'{"dataframe_split": {"columns":["CODE_GENDER_M","EXT_SOURCE_3","EXT_SOURCE_2","NAME_EDUCATION_TYPE_Secondary_or_secondary_special","NAME_EDUCATION_TYPE_Higher_education","NAME_CONTRACT_TYPE_Cash_loans","NAME_INCOME_TYPE_Working"],"index":[0],"data":[[1,0.2020866017,0.5806283659,1,0,1,1]]}}' 


Copier cette ligne de commande Linux sur un terminal local et vérifier qu'elle renvoie une cible prédite = 1 :

In [28]:
print('curl -d' + str_data + '''-H 'Content-Type: application/json' -X POST ''' + url_staging)

curl -d'{"dataframe_split": {"columns":["CODE_GENDER_M","EXT_SOURCE_3","EXT_SOURCE_2","NAME_EDUCATION_TYPE_Secondary_or_secondary_special","NAME_EDUCATION_TYPE_Higher_education","NAME_CONTRACT_TYPE_Cash_loans","NAME_INCOME_TYPE_Working"],"index":[0],"data":[[1,0.2020866017,0.5806283659,1,0,1,1]]}}' -H 'Content-Type: application/json' -X POST 4.233.201.217:5677/invocations


# Serveur de production

Le déploiment sur le serveur de Production a besoin pour se déclencher de :
- la publication via _git-push_ d'une nouvelle version du modèle vers le serveur de Staging
- la réussite des tests unitaires 

In [29]:
dir_production = './production_model'
port_production = dict_config['port_production']
! mkdir -p $dir_production

## Chargement du modèle

In [37]:
# ! cp ./staging_model/* ./production_model/

In [38]:
production_model = mlflow.pyfunc.load_model(dir_production)
production_model

mlflow.pyfunc.loaded_model:
  flavor: mlflow.sklearn

**Vérification de la signature**

La signature du modèle chargé indique les valeurs valides à lui fournir

Attributs

In [39]:
li_features = production_model.metadata.get_input_schema().input_names()
li_features

['CODE_GENDER_M',
 'EXT_SOURCE_3',
 'EXT_SOURCE_2',
 'NAME_EDUCATION_TYPE_Secondary_or_secondary_special',
 'NAME_EDUCATION_TYPE_Higher_education',
 'NAME_CONTRACT_TYPE_Cash_loans',
 'NAME_INCOME_TYPE_Working']

Cible

In [40]:
production_model.metadata.get_output_schema().input_names

<bound method Schema.input_names of ['TARGET': long (required)]>

**Vérification de prédiction**

On prédit la cible pour une observation TP

In [41]:
production_model.predict(df_TP_sample_1)

array([1])

## Lancement du serveur

**Arrêt du serveur**

In [42]:
mask = ':' + port_production
! pkill -f "$mask"

**Démarrage**

In [43]:
ip_host, port_production = '0.0.0.0', '5678'
shell_command = 'mlflow models serve -m ' + dir_production + ' -p ' + port_production + ' -h ' + ip_host
get_ipython().system_raw(shell_command + ' --no-conda &')          # runs model API in background

**Vérification d'exécution**

Il y a 2 processus qui tournent par serveur

In [44]:
! ps aux | grep "scoring_server" | grep -v "grep" | awk '{print $2, $15, $19}'

70156 0.0.0.0:5677 mlflow.pyfunc.scoring_server.wsgi:app
70157 0.0.0.0:5677 mlflow.pyfunc.scoring_server.wsgi:app


2024/07/02 22:50:25 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'
2024/07/02 22:50:25 INFO mlflow.pyfunc.backend: === Running command 'exec gunicorn --timeout=60 -b 0.0.0.0:5678 -w 1 ${GUNICORN_CMD_ARGS} -- mlflow.pyfunc.scoring_server.wsgi:app'
[2024-07-02 22:50:25 +0000] [70505] [INFO] Starting gunicorn 22.0.0
[2024-07-02 22:50:25 +0000] [70505] [INFO] Listening at: http://0.0.0.0:5678 (70505)
[2024-07-02 22:50:25 +0000] [70505] [INFO] Using worker: sync
[2024-07-02 22:50:25 +0000] [70506] [INFO] Booting worker with pid: 70506


## Configuration réseau
### Tunnel SSH
Sur le terminal [_Azure CLI_](https://portal.azure.com/#cloudshell/) :

In [45]:
shell_command = 'ssh -v -N -L ' + port_production + ':localhost:' + port_production + ' azureuser@' + ip_server
print(shell_command)

ssh -v -N -L 5678:localhost:5678 azureuser@4.233.201.217


### Aperture de ports
Sur le [portail _Azure_](https://portal.azure.com/), _Network Settings_ de la VM : _Create Inbound Port Rule_

In [46]:
print('Destination Port =', port_production)

Destination Port = 5678


## URL de l'API

In [47]:
url_production = ip_server + ':' + port_production + '/invocations'
print('URL Production    -> http://' + url_production)

URL Production    -> http://4.233.201.217:5678/invocations


## Vérification d'accès distant

Demande par requête POST de prédiction de la cible pour une observation

In [48]:
str_features_values = df_TP_sample_1.to_json(orient='split')
print(str_features_values)

{"columns":["CODE_GENDER_M","EXT_SOURCE_3","EXT_SOURCE_2","NAME_EDUCATION_TYPE_Secondary_or_secondary_special","NAME_EDUCATION_TYPE_Higher_education","NAME_CONTRACT_TYPE_Cash_loans","NAME_INCOME_TYPE_Working"],"index":[0],"data":[[1,0.2020866017,0.5806283659,1,0,1,1]]}


In [49]:
str_data = '\'{"dataframe_split": ' + str_features_values + '}\' '
print(str_data)

'{"dataframe_split": {"columns":["CODE_GENDER_M","EXT_SOURCE_3","EXT_SOURCE_2","NAME_EDUCATION_TYPE_Secondary_or_secondary_special","NAME_EDUCATION_TYPE_Higher_education","NAME_CONTRACT_TYPE_Cash_loans","NAME_INCOME_TYPE_Working"],"index":[0],"data":[[1,0.2020866017,0.5806283659,1,0,1,1]]}}' 


Copier cette ligne de commande Linux sur un terminal local :

In [50]:
print('curl -d' + str_data + '''-H 'Content-Type: application/json' -X POST ''' + url_production)

curl -d'{"dataframe_split": {"columns":["CODE_GENDER_M","EXT_SOURCE_3","EXT_SOURCE_2","NAME_EDUCATION_TYPE_Secondary_or_secondary_special","NAME_EDUCATION_TYPE_Higher_education","NAME_CONTRACT_TYPE_Cash_loans","NAME_INCOME_TYPE_Working"],"index":[0],"data":[[1,0.2020866017,0.5806283659,1,0,1,1]]}}' -H 'Content-Type: application/json' -X POST 4.233.201.217:5678/invocations


[2024-07-02 23:05:22 +0000] [70156] [INFO] Handling signal: term
[2024-07-02 23:05:22 +0000] [70157] [INFO] Worker exiting (pid: 70157)
[2024-07-02 23:05:22 +0000] [70156] [INFO] Shutting down: Master
[2024-07-02 23:05:35 +0000] [70505] [INFO] Handling signal: term
[2024-07-02 23:05:35 +0000] [70506] [INFO] Worker exiting (pid: 70506)
[2024-07-02 23:05:36 +0000] [70505] [INFO] Shutting down: Master


<a name="Current_Cell"></a>
<hr color="red" size=5>

# Fin du traitement

In [ ]:
assert(False) # prevents the execution of following cells

# Annexes

## Déploiement via MLFlow UI
La version du modèle peut être mise à disposition par _MLFlow UI_ afin d'être déployée 

**Instructions sous _MLFlow UI_ :**
1. click sur le _run_ du modèle à déployer
2. onglet "_Artifacts_", click sur le bouton "_Register_"
    - nommer le modèle à déployer "ml_model_to_deploy"
3. lancer le service du modèle 

## Référentiel d'évaluation
Remarque : le chiffre des dizaines a été ajouté aux références CE originales pour mieux les différencier

Définir la stratégie d’élaboration d’un modèle d’apprentissage supervisé et sélectionner et entraîner des modèles adaptés à une problématique métier afin de réaliser une analyse prédictive.
CE1 Les variables catégorielles identifiées ont été transformées en fonction du besoin (par exemple via OneHotEncoder ou TargetEncoder).

CE2 Vous avez a créé de nouvelles variables à partir de variables existantes.

CE3 Vous avez réalisé des transformations mathématiques lorsque c'est requis pour transformer les distributions de variables.

CE4 Vous avez normalisé les variables lorsque c'est requis.

CE5 Vous avez défini sa stratégie d’élaboration d’un modèle pour répondre à un besoin métier. Cela signifie dans ce projet que :

l’étudiant a présenté son approche méthodologique de modélisation dans son support de présentation pendant la soutenance et est capable de répondre à des questions à ce sujet, si elles lui sont posées.
CE6 Vous avez choisi la ou les variables cibles pertinentes.

CE7 Vous avez vérifié qu'il n’y a pas de problème de data leakage (c'est-à-dire, des variables trop corrélées à la variable cible et inconnues a priori dans les données en entrée du modèle).

CE8 Vous avez testé plusieurs algorithmes de façon cohérente, en partant des plus simples vers les plus complexes (au minimum un linéaire et un non linéaire).



Évaluer les performances des modèles d’apprentissage supervisé selon différents critères (scores, temps d'entraînement, etc.) en adaptant les paramètres afin de choisir le modèle le plus performant pour la problématique métier.

CE1 Vous avez choisi une métrique adaptée pour évaluer la performance d'un algorithme (par exemple : R2 ou RMSE en régression, accuracy ou AUC en classification, etc.). Dans le cadre de ce projet, cela signifie que :

Vous avez mis en oeuvre un score métier pour évaluer les modèles et optimiser les hyperparamètres, qui prend en compte les spécificités du contexte, en particulier le fait que le coût d’un faux négatif et d’un faux positif sont sensiblement différents.
CE2 Vous avez exploré d'autres indicateurs de performance que le score pour comprendre les résultats (coefficients des variables en fonction de la pénalisation, visualisation des erreurs en fonction des variables du modèle, temps de calcul...).

CE3 Vous avez séparé les données en train/test pour les évaluer de façon pertinente et détecter l'overfitting.

CE4 Vous avez mis en place un modèle simple de référence pour évaluer le pouvoir prédictif du modèle choisi (dummyRegressor ou dummyClassifier).

CE5 Vous avez pris en compte dans sa démarche de modélisation l'éventuel déséquilibre des classes (dans le cas d'une classification).

CE6 Vous avez optimisé les hyper-paramètres pertinents dans les différents algorithmes.

CE7 Vous avez mis en place une validation croisée (via GridsearchCV, RandomizedSearchCV ou équivalent) afin d’optimiser les hyperparamètres et comparer les modèles. Dans le cadre de ce projet :

une cross-validation du dataset train est réalisée ;
un premier test de différentes valeurs d’hyperparamètres est réalisé sur chaque algorithme testé, et affiné pour l’algorithme final choisi ;
tout projet présentant un score AUC anormalement élevé, démontrant de l’overfitting dans le GrisSearchCV, sera invalidé (il ne devrait pas être supérieur au meilleur de la compétition Kaggle : 0.82).
CE8 Vous avez présenté l'ensemble des résultats en allant des modèles les plus simples aux plus complexes. Vous avez justifié le choix final de l'algorithme et des hyperparamètres.

CE9 Vous avez réalisé l’analyse de l’importance des variables (feature importance) globale sur l’ensemble du jeu de données et locale sur chaque individu du jeu de données.



Définir et mettre en œuvre un pipeline d’entraînement des modèles, avec centralisation du stockage des modèles et formalisation des résultats et mesures des différentes expérimentations réalisées, afin d’industrialiser le projet de Machine Learning.
CE1 Vous avez mis en oeuvre un pipeline d’entraînement des modèles reproductible.

CE2 Vous avez sérialisé et stocké les modèles créés dans un registre centralisé afin de pouvoir facilement les réutiliser.

CE3 Vous avez formalisé des mesures et résultats de chaque expérimentation, afin de les analyser et de les comparer



Mettre en œuvre un logiciel de version de code afin d’assurer en continu l’intégration et la diffusion du modèle auprès de collaborateurs.
CE1 Vous avez créé un dossier contenant tous les scripts du projet dans un logiciel de version de code avec Git et l'a partagé avec Github.

CE2 Vous avez présenté un historique des modifications du projet qui affiche au moins trois versions distinctes, auxquelles il est possible d'accéder.

CE3 Vous avez tenu à jour et mis à disposition la liste des packages utilisés ainsi que leur numéro de version.

CE4 Vous avez rédigé un fichier introductif permettant de comprendre l'objectif du projet et le découpage des dossiers.

CE5 Vous avez commenté les scripts et les fonctions facilitant une réutilisation du travail par d'autres personnes et la collaboration.



Concevoir et assurer un déploiement continu d'un moteur d’inférence (modèle de prédiction encapsulé dans une API) sur une plateforme Cloud afin de permettre à des applications de réaliser des prédictions via une requête à l’API.

CE1 Vous avez défini et préparé un pipeline de déploiement continu.

CE2 Vous avez déployé le modèle de machine learning sous forme d'API (via Flask par exemple) et cette API renvoie bien une prédiction correspondant à une demande.

CE3 Vous avez mis en œuvre un pipeline de déploiement continu, afin de déployer l'API sur un serveur d'une plateforme Cloud.

CE4 Vous avez mis en oeuvre des tests unitaires automatisés (par exemple avec pyTest).

CE5 Vous avez réalisé l'API indépendamment de l'application qui utilise le résultat de la prédiction.



Définir et mettre en œuvre une stratégie de suivi de la performance d’un modèle en production et en assurer la maintenance afin de garantir dans le temps la production de prédictions performantes.

CE1 Vous avez défini une stratégie de suivi de la performance du modèle. Dans le cadre du projet :
choix de réaliser a priori cette analyse sur le dataset disponible : analyse de data drift entre le dataset train et le dataset test.

CE2 Vous avez réalisé un système de stockage d’événements relatifs aux prédictions réalisées par l’API et une gestion d’alerte en cas de dégradation significative de la performance. Dans le cadre du projet :
choix de réaliser a priori cette analyse analyse de data drift, via une simulation dans un notebook et création d’un tableau HTML d’analyse avec la librairie evidently.

CE3 Vous avez analysé la stabilité du modèle dans le temps et défini des actions d’amélioration de sa performance. Dans le cadre de ce projet :
analyse du tableau HTML evidently, et conclusion sur un éventuel data drift.